In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer
import numpy as np
from pathlib import Path
import json
import torch
import polars as pl
#If you already saved the model locally and are using docker
client = QdrantClient(host="localhost", port=6333, prefer_grpc=True, timeout=1000)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("models/multilingual-e5-large", device=device)

# Run docker container before
# docker run -d --name qdrant -p 6333:6333 -p 6334:6334 -v /home/Loris/EPFL/MA3/ML/project2/project2Rag/qdrant_storage:/qdrant/storage qdrant/qdrant:latest

# VERY IMPORTANT : to query the remote qdrand docker container over ssh, run the following in a terminale before:
# ssh -L 6333:localhost:6333 -L 6334:localhost:6334 <your username>@dclgpusrv.epfl.ch -N
#Make sure to have your ssh keys before
#This forwards the qdrant ports to your local machine
#If you are outside EPFL network, use VPN

/localhome/ltran/.conda/envs/DCL_WIKI_RAG/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from qdrant_client.http import models as rest
#get articles count from Qdrant DB 

collection_name = "wikipedia_fr_chunks"
total = client.count(collection_name=collection_name, exact=True).count
print(f"Total articles stored: {total}")

Total articles stored: 2800041


In [3]:
import polars as pl
import numpy as np
from tqdm import tqdm

# Keep as LazyFrame - don't collect the full dataset!
df_lazy = pl.scan_parquet("articles_fr_merged.parquet").filter(
    pl.col('link_count') > 0
)

# Only collect a small sample to verify structure
sample = df_lazy.head(1).collect()

print(f"📚 Dataset loaded as LazyFrame (not in memory)")
print(f"   Schema: {df_lazy.collect_schema()}")
print(f"\n   Link structure example:")


full_text = df_lazy.select("text").head(1).collect().item()

print(full_text)

📚 Dataset loaded as LazyFrame (not in memory)
   Schema: Schema({'id': Int64, 'title': String, 'text': String, 'links': List(Struct({'full_url': String, 'start_idx': Int64, 'anchor': String, 'href_raw': String, 'href_decoded': String})), 'link_count': UInt32, 'text_withoutHref': String})

   Link structure example:
Antoine Meillet, né le à &lt;a href="Moulins%20%28Allier%29"&gt;Moulins&lt;/a&gt; (&lt;a href="Allier%20%28d%C3%A9partement%29"&gt;Allier&lt;/a&gt;) et mort le à &lt;a href="Ch%C3%A2teaumeillant"&gt;Châteaumeillant&lt;/a&gt; (&lt;a href="Cher%20%28d%C3%A9partement%29"&gt;Cher&lt;/a&gt;), est un &lt;a href="Philologie"&gt;philologue&lt;/a&gt; français, le principal &lt;a href="liste%20de%20linguistes"&gt;linguiste&lt;/a&gt; français des premières décennies du XX siècle.
Biographie.
Enfance et formation.
Paul Jules Antoine Meillet est d'origine &lt;a href="Allier%20%28d%C3%A9partement%29"&gt;bourbonnaise&lt;/a&gt;, fils d'un &lt;a href="notaire"&gt;notaire&lt;/a&gt; de &lt;a h

In [18]:
# Check payload structure of new collection
sample = client.scroll(
    collection_name="wikipedia_fr_chunks",
    limit=1,
    with_payload=True,
    with_vectors=False
)[0]
print(sample[0].payload.keys())

dict_keys(['paragraph_index', 'word_count', 'source_article_id', 'chunk_type', 'sentence_group_index', 'source_article_title', 'global_chunk_id', 'text_preview', 'text'])


In [19]:
# import torch
# import re
# from typing import List, Dict, Tuple
# from urllib.parse import unquote

# def create_url_to_id_mapping_from_qdrant(client, collection_name: str = "wikipedia_fr_chunks") -> Dict[str, int]:
#     """Build URL → ID mapping using articles in Qdrant."""
#     from urllib.parse import quote
    
#     url_to_id = {}
#     id_to_title = {}
    
#     print(f"🗺️  Building URL → ID mapping from '{collection_name}'...")
    
#     offset = None
#     batch_size = 1000
#     total_processed = 0
    
#     while True:
#         points, offset = client.scroll(
#             collection_name=collection_name,
#             limit=batch_size,
#             offset=offset,
#             with_payload=True,
#             with_vectors=False
#         )
        
#         if not points:
#             break
        
#         for point in points:
#             article_id = point.payload.get("id")
#             title = point.payload.get("title", "")
            
#             if not article_id or not title:
#                 continue
            
#             id_to_title[article_id] = title
            
#             # Create URL pattern variations
#             patterns = [
#                 title,
#                 title.replace(" ", "_"),
#                 quote(title.replace(" ", "_"), safe=""),
#                 title.lower(),
#                 title.lower().replace(" ", "_"),
#                 title.lower().replace(" ", "%20"),
#             ]
            
#             for pattern in patterns:
#                 url_to_id[pattern] = article_id
        
#         total_processed += len(points)
        
#         if offset is None:
#             break
    
#     print(f"✅ Created {len(url_to_id):,} URL mappings from {total_processed:,} articles")
#     return url_to_id, id_to_title

# # Build mappings
# url_to_id, id_to_title = create_url_to_id_mapping_from_qdrant(client, "wikipedia_fr_chunks")

import torch
import re
from typing import List, Dict, Tuple
from urllib.parse import unquote

def create_url_to_id_mapping_from_qdrant(client, collection_name: str = "wikipedia_fr_chunks") -> Dict[str, int]:
    """Build URL → ID mapping using articles in Qdrant."""
    from urllib.parse import quote
    
    url_to_id = {}
    id_to_title = {}
    
    print(f"🗺️  Building URL → ID mapping from '{collection_name}'...")
    
    offset = None
    batch_size = 1000
    total_processed = 0
    seen_article_ids = set()  # Track unique articles (chunks share same source)
    
    while True:
        points, offset = client.scroll(
            collection_name=collection_name,
            limit=batch_size,
            offset=offset,
            with_payload=True,
            with_vectors=False
        )
        
        if not points:
            break
        
        for point in points:
            # === FIX: Use correct field names for chunked collection ===
            article_id = point.payload.get("source_article_id")  # Changed from "id"
            title = point.payload.get("source_article_title", "")  # Changed from "title"
            
            if not article_id or not title:
                continue
            
            # Skip if we've already processed this article (multiple chunks per article)
            if article_id in seen_article_ids:
                continue
            seen_article_ids.add(article_id)
            
            id_to_title[article_id] = title
            
            # Create URL pattern variations
            patterns = [
                title,
                title.replace(" ", "_"),
                quote(title.replace(" ", "_"), safe=""),
                title.lower(),
                title.lower().replace(" ", "_"),
                title.lower().replace(" ", "%20"),
            ]
            
            for pattern in patterns:
                url_to_id[pattern] = article_id
        
        total_processed += len(points)
        
        if offset is None:
            break
    
    print(f"✅ Created {len(url_to_id):,} URL mappings from {len(seen_article_ids):,} unique articles ({total_processed:,} chunks)")
    return url_to_id, id_to_title

# Build mappings
url_to_id, id_to_title = create_url_to_id_mapping_from_qdrant(client, "wikipedia_fr_chunks")

🗺️  Building URL → ID mapping from 'wikipedia_fr_chunks'...
✅ Created 11,095,075 URL mappings from 2,230,038 unique articles (2,800,041 chunks)


In [20]:
import pickle

# Save mappings to disk
with open("url_to_id_mapping.pkl", "wb") as f:
    pickle.dump(url_to_id, f)

with open("id_to_title_mapping.pkl", "wb") as f:
    pickle.dump(id_to_title, f)

print(f"✅ Saved {len(url_to_id):,} URL mappings to url_to_id_mapping.pkl")
print(f"✅ Saved {len(id_to_title):,} ID→title mappings to id_to_title_mapping.pkl")

✅ Saved 11,095,075 URL mappings to url_to_id_mapping.pkl
✅ Saved 2,230,038 ID→title mappings to id_to_title_mapping.pkl


In [4]:
import pickle
from pathlib import Path

# Load from disk if available, otherwise rebuild
if Path("url_to_id_mapping.pkl").exists() and Path("id_to_title_mapping.pkl").exists():
    print("📂 Loading cached mappings...")
    with open("url_to_id_mapping.pkl", "rb") as f:
        url_to_id = pickle.load(f)
    with open("id_to_title_mapping.pkl", "rb") as f:
        id_to_title = pickle.load(f)
    print(f"✅ Loaded {len(url_to_id):,} URL mappings, {len(id_to_title):,} ID→title mappings")
else:
    print("⚠️ No cached mappings found, rebuilding...")
    url_to_id, id_to_title = create_url_to_id_mapping_from_qdrant(client, "wikipedia_fr")

📂 Loading cached mappings...
✅ Loaded 11,095,075 URL mappings, 2,230,038 ID→title mappings


In [5]:
import re
import polars as pl
from urllib.parse import unquote
from typing import List, Dict, Tuple

def extract_links_and_clean_text(text: str) -> Tuple[str, List[Dict]]:
    """
    Extract href links from text and return clean text with anchors only.
    
    Input text format:
    'Antoine Meillet, né le à <a href="Moulins%20%28Allier%29">Moulins</a>...'
    
    Returns:
    - clean_text: 'Antoine Meillet, né le à Moulins...'
    - links: [{'anchor': 'Moulins', 'href_raw': 'Moulins%20%28Allier%29', 
               'href_decoded': 'Moulins (Allier)', 'start_idx': 25, 'end_idx': 32}]
    """
    if not text:
        return "", []
    
    # Pattern to match <a href="...">anchor</a>
    # Handles both &lt;a href=... and <a href=... formats
    href_pattern = r'(?:&lt;|<)a\s+href="([^"]*)"(?:&gt;|>)(.*?)(?:&lt;|<)/a(?:&gt;|>)'
    
    links = []
    clean_text = text
    offset = 0  # Track position offset as we replace tags
    
    for match in re.finditer(href_pattern, text, re.IGNORECASE | re.DOTALL):
        href_raw = match.group(1)
        anchor = match.group(2)
        
        # Decode URL-encoded href
        try:
            href_decoded = unquote(href_raw)
        except:
            href_decoded = href_raw
        
        # Calculate position in clean text
        # Original position minus the offset from previous replacements
        original_start = match.start()
        clean_start = original_start - offset
        
        # Store link info
        links.append({
            'anchor': anchor,
            'href_raw': href_raw,
            'href_decoded': href_decoded,
            'start_idx': clean_start,
            'end_idx': clean_start + len(anchor)
        })
        
        # Update offset: we're removing the full tag and keeping only anchor
        tag_length = match.end() - match.start()
        anchor_length = len(anchor)
        offset += tag_length - anchor_length
    
    # Replace all href tags with just the anchor text
    clean_text = re.sub(href_pattern, r'\2', text, flags=re.IGNORECASE | re.DOTALL)
    
    return clean_text, links


def process_text_column(text: str) -> Dict:
    """
    Process a single text entry and return clean text + links.
    For use with Polars map_elements.
    """
    clean_text, links = extract_links_and_clean_text(text)
    return {
        'text_clean': clean_text,
        'extracted_links': links
    }


# Create new LazyFrame with extracted links and clean text
# This processes the 'text' column which contains the href tags

df_with_links = (
    df_lazy
    .with_columns([
        # Extract links and clean text using map_elements
        pl.col('text').map_elements(
            lambda x: extract_links_and_clean_text(x)[0] if x else "",
            return_dtype=pl.Utf8
        ).alias('text_clean'),
        
        pl.col('text').map_elements(
            lambda x: extract_links_and_clean_text(x)[1] if x else [],
            return_dtype=pl.List(pl.Struct([
                pl.Field('anchor', pl.Utf8),
                pl.Field('href_raw', pl.Utf8),
                pl.Field('href_decoded', pl.Utf8),
                pl.Field('start_idx', pl.Int64),
                pl.Field('end_idx', pl.Int64)
            ]))
        ).alias('extracted_links')
    ])
)

# Test on a sample
sample = df_with_links.head(1).collect()
print("Sample result:")
print(f"Original text (first 200 chars): {sample['text'][0][:200]}...")
print(f"\nClean text (first 200 chars): {sample['text_clean'][0][:200]}...")
print(f"\nExtracted links ({len(sample['extracted_links'][0])} total):")
for link in sample['extracted_links'][0][:5]:
    print(f"  - '{link['anchor']}' → {link['href_decoded']} (pos: {link['start_idx']})")

Sample result:
Original text (first 200 chars): Antoine Meillet, né le à &lt;a href="Moulins%20%28Allier%29"&gt;Moulins&lt;/a&gt; (&lt;a href="Allier%20%28d%C3%A9partement%29"&gt;Allier&lt;/a&gt;) et mort le à &lt;a href="Ch%C3%A2teaumeillant"&gt;C...

Clean text (first 200 chars): Antoine Meillet, né le à Moulins (Allier) et mort le à Châteaumeillant (Cher), est un philologue français, le principal linguiste français des premières décennies du XX siècle.
Biographie.
Enfance et ...

Extracted links (65 total):
  - 'Moulins' → Moulins (Allier) (pos: 25)
  - 'Allier' → Allier (département) (pos: 34)
  - 'Châteaumeillant' → Châteaumeillant (pos: 55)
  - 'Cher' → Cher (département) (pos: 72)
  - 'philologue' → Philologie (pos: 86)


In [6]:
import re

# def extract_sentences_with_links_by_anchor(
#     text: str,
#     links: list
# ) -> List[Dict]:
#     """Extract sentences and match links by anchor text presence (whole word match)."""
#     if not text or not links or len(links) == 0:
#         return []
    
#     # Split into sentences
#     sentence_pattern = r'(?<=[.!?])\s+'
#     sentences = re.split(sentence_pattern, text)
    
#     results = []
    
#     for sentence in sentences:
#         if not sentence.strip() or len(sentence) < 20:
#             continue
        
#         links_in_sent = []
#         seen_links = set()
        
#         for link in links:
#             anchor = link.get('anchor', '')
#             href_decoded = link.get('href_decoded', '')
            
#             if not anchor or len(anchor) < 2:
#                 continue
            
#             # Use word boundary matching to avoid substring matches
#             # Escape special regex characters in anchor
#             escaped_anchor = re.escape(anchor)
#             # Match as whole word (with word boundaries)
#             pattern = r'\b' + escaped_anchor + r'\b'
            
#             if re.search(pattern, sentence, re.IGNORECASE):
#                 link_key = (anchor.lower(), href_decoded.lower())
#                 if link_key not in seen_links:
#                     seen_links.add(link_key)
#                     links_in_sent.append({
#                         'anchor': anchor,
#                         'href_decoded': href_decoded,
#                     })
        
#         if links_in_sent:
#             results.append({
#                 'sentence': sentence.strip(),
#                 'links_in_sentence': links_in_sent,
#                 'num_links': len(links_in_sent)
#             })
    
#     return results

def extract_sentences_with_links_by_position(
    clean_text: str,
    extracted_links: List[Dict]
) -> List[Dict]:
    """
    Extract sentences and match links by their character position in clean text.
    
    This ensures only links that were ACTUALLY in that sentence (in the original
    Wikipedia markup) are included as ground truth.
    """
    if not clean_text or not extracted_links or len(extracted_links) == 0:
        return []
    
    # Sort links by position
    sorted_links = sorted(extracted_links, key=lambda x: x.get('start_idx', 0))
    
    # Split into sentences
    sentence_pattern = r'(?<=[.!?])\s+'
    sentences = re.split(sentence_pattern, clean_text)
    
    results = []
    current_pos = 0
    
    for sentence in sentences:
        if not sentence.strip() or len(sentence) < 20:
            # Still need to track position even for skipped sentences
            idx = clean_text.find(sentence, current_pos)
            if idx != -1:
                current_pos = idx + len(sentence)
            continue
        
        # Find sentence boundaries in clean text
        sent_start = clean_text.find(sentence, current_pos)
        if sent_start == -1:
            continue
        sent_end = sent_start + len(sentence)
        current_pos = sent_end
        
        # Find all links whose position falls within this sentence
        links_in_sent = []
        for link in sorted_links:
            link_start = link.get('start_idx', -1)
            link_end = link.get('end_idx', -1)
            
            # Check if link position falls within sentence boundaries
            if sent_start <= link_start < sent_end:
                links_in_sent.append({
                    'anchor': link.get('anchor', ''),
                    'href_decoded': link.get('href_decoded', ''),
                    'href_raw': link.get('href_raw', ''),
                    'position': link_start
                })
        
        if links_in_sent:
            results.append({
                'sentence': sentence.strip(),
                'links_in_sentence': links_in_sent,
                'num_links': len(links_in_sent),
                'start_pos': sent_start,
                'end_pos': sent_end
            })
    
    return results

In [7]:
from nltk.tokenize import sent_tokenize
import nltk
# nltk.download('punkt')  # Run once if needed

def paragraph_chunk(text: str, min_length: int = 50) -> List[str]:
    """
    Split text on paragraph breaks (double newlines).
    Filter out very short paragraphs.
    """
    paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
    # Also split on single newlines followed by headers (common in Wikipedia)
    result = []
    for p in paragraphs:
        # Split further on section headers (lines ending with period or colon followed by newline)
        sub_parts = re.split(r'\n(?=[A-Z])', p)
        for part in sub_parts:
            if len(part.strip()) >= min_length:
                result.append(part.strip())
    return result


def sentence_chunk(text: str, max_words: int = 150, min_words: int = 20) -> List[str]:
    """
    Group sentences into chunks under a specified word count.
    Preserves complete sentences.
    """
    try:
        sentences = sent_tokenize(text, language='french')
    except:
        # Fallback to simple regex if NLTK fails
        sentences = re.split(r'(?<=[.!?])\s+', text)
    
    chunks = []
    buffer = []
    length = 0
    
    for sent in sentences:
        count = len(sent.split())
        if length + count > max_words and buffer:
            chunk_text = " ".join(buffer)
            if len(chunk_text.split()) >= min_words:
                chunks.append(chunk_text)
            buffer = []
            length = 0
        buffer.append(sent)
        length += count
    
    # Don't forget the last buffer
    if buffer:
        chunk_text = " ".join(buffer)
        if len(chunk_text.split()) >= min_words:
            chunks.append(chunk_text)
    
    return chunks


def recursive_chunk(text: str, chunk_size: int = 512, overlap: int = 50) -> List[str]:
    """
    Recursive chunking: try paragraphs first, then sentences, then words.
    """
    # First try paragraph-based
    paragraphs = paragraph_chunk(text, min_length=50)
    
    chunks = []
    for para in paragraphs:
        word_count = len(para.split())
        if word_count <= chunk_size:
            chunks.append(para)
        else:
            # Paragraph too long, split into sentences
            sentence_chunks = sentence_chunk(para, max_words=chunk_size)
            chunks.extend(sentence_chunks)
    
    return chunks

In [ ]:
# # import torch
# # import re
# # from typing import List, Dict, Tuple
# # from urllib.parse import unquote

# # def create_test_dataset_optimized(
# #     df_lazy: pl.LazyFrame,
# #     url_to_id: Dict[str, int],
# #     max_sentences: int = 100,
# #     min_links_per_sentence: int = 1,
# #     sample_articles: int = 5000
# # ) -> List[Dict]:
# #     """
# #     Create test dataset using LazyFrame streaming.
# #     """
# #     print(f"🔍 Creating test dataset (max {max_sentences} sentences)...")
# #     print(f"   Sampling {sample_articles:,} articles from LazyFrame...")
    
# #     sampled_df = (
# #         df_lazy
# #         .filter(pl.col('link_count') > 0)
# #         .filter(pl.col('text_withoutHref').is_not_null())
# #         .filter(pl.col('text_withoutHref').str.len_chars() > 100)
# #         .select(['id', 'title', 'text_withoutHref', 'links'])
# #         .head(sample_articles * 3)
# #         .collect(streaming=True)
# #         .sample(n=min(sample_articles, sample_articles * 3), shuffle=True, seed=42)
# #     )
    
# #     print(f"   Loaded {len(sampled_df):,} articles into memory")
    
# #     test_data = []
# #     articles_processed = 0
    
# #     for row in tqdm(sampled_df.iter_rows(named=True), total=len(sampled_df), desc="Processing"):
# #         if len(test_data) >= max_sentences:
# #             break
        
# #         article_id = row["id"]
# #         article_title = row["title"]
# #         text = row.get("text_withoutHref", "")
# #         links_raw = row.get("links", [])
        
# #         if links_raw is None:
# #             continue
# #         links = list(links_raw) if hasattr(links_raw, '__iter__') else []
        
# #         if not text or len(links) == 0:
# #             continue
        
# #         # Use anchor-based matching instead of position-based
# #         sentences_with_links = extract_sentences_with_links_by_anchor(text, links)
        
# #         for sent_data in sentences_with_links:
# #             if len(test_data) >= max_sentences:
# #                 break
            
# #             sentence = sent_data["sentence"]
# #             links_in_sent = sent_data["links_in_sentence"]
            
# #             # Verify anchor is actually in sentence
# #             ground_truth_links = []
# #             for link in links_in_sent:
# #                 href_decoded = link["href_decoded"]
# #                 anchor = link["anchor"]
                
# #                 # Double-check anchor is in sentence
# #                 if anchor.lower() not in sentence.lower():
# #                     continue
                
# #                 if not anchor or not anchor.strip():
# #                     continue
                
# #                 target_id = url_to_id.get(href_decoded)
# #                 if not target_id:
# #                     for variation in [
# #                         href_decoded.replace("_", " "),
# #                         href_decoded.replace("%20", " "),
# #                         href_decoded.lower(),
# #                         href_decoded.lower().replace("_", " ")
# #                     ]:
# #                         target_id = url_to_id.get(variation)
# #                         if target_id:
# #                             break
                
# #                 if target_id and target_id != article_id:
# #                     ground_truth_links.append({
# #                         'anchor': anchor,
# #                         'target_id': target_id,
# #                         'href_decoded': href_decoded
# #                     })
            
# #             if len(ground_truth_links) >= min_links_per_sentence:
# #                 test_data.append({
# #                     'source_article_id': article_id,
# #                     'source_article_title': article_title,
# #                     'sentence': sentence,
# #                     'ground_truth_links': ground_truth_links,
# #                     'num_ground_truth': len(ground_truth_links)
# #                 })
        
# #         articles_processed += 1
    
# #     del sampled_df
# #     import gc
# #     gc.collect()
    
# #     print(f"\n✅ Created test dataset:")
# #     print(f"   Sentences: {len(test_data)}")
# #     print(f"   Articles processed: {articles_processed}")
# #     total_gt_links = sum(t['num_ground_truth'] for t in test_data)
# #     print(f"   Total ground truth links: {total_gt_links}")
    
# #     return test_data

# # # Create test dataset - pass LazyFrame, not collected DataFrame
# # test_data = create_test_dataset_optimized(
# #     df_lazy,  # LazyFrame, not df
# #     url_to_id, 
# #     max_sentences=1000000,
# #     min_links_per_sentence=20,
# #     sample_articles=1000  # Only load 5K articles into memory
# # )


# def create_test_dataset_with_position_matching(
#     df_lazy: pl.LazyFrame,
#     url_to_id: Dict[str, int],
#     max_sentences: int = 100,
#     min_links_per_sentence: int = 1,
#     sample_articles: int = 5000
# ) -> List[Dict]:
#     """
#     Create test dataset using position-based link matching from raw text.
#     """
#     print(f"🔍 Creating test dataset with position-based matching...")
#     print(f"   Sampling {sample_articles:,} articles...")
    
#     # Sample articles and process text column
#     sampled_df = (
#         df_lazy
#         .filter(pl.col('link_count') > 0)
#         .filter(pl.col('text').is_not_null())
#         .filter(pl.col('text').str.len_chars() > 100)
#         .select(['id', 'title', 'text'])
#         .head(sample_articles * 3)
#         .collect(streaming=True)
#         .sample(n=min(sample_articles, sample_articles * 3), shuffle=True, seed=42)
#     )
    
#     print(f"   Loaded {len(sampled_df):,} articles")
    
#     test_data = []
#     articles_processed = 0
    
#     for row in tqdm(sampled_df.iter_rows(named=True), total=len(sampled_df), desc="Processing"):
#         if len(test_data) >= max_sentences:
#             break
        
#         article_id = row["id"]
#         article_title = row["title"]
#         raw_text = row.get("text", "")
        
#         if not raw_text:
#             continue
        
#         # Extract links and get clean text
#         clean_text, extracted_links = extract_links_and_clean_text(raw_text)
        
#         if not clean_text or len(extracted_links) == 0:
#             continue
        
#         # Extract sentences with position-based link matching
#         sentences_with_links = extract_sentences_with_links_by_position(clean_text, extracted_links)
        
#         for sent_data in sentences_with_links:
#             if len(test_data) >= max_sentences:
#                 break
            
#             sentence = sent_data["sentence"]
#             links_in_sent = sent_data["links_in_sentence"]
            
#             # Map links to article IDs
#             ground_truth_links = []
#             for link in links_in_sent:
#                 href_decoded = link["href_decoded"]
#                 anchor = link["anchor"]
                
#                 if not anchor or not anchor.strip():
#                     continue
                
#                 target_id = url_to_id.get(href_decoded)
#                 if not target_id:
#                     for variation in [
#                         href_decoded.replace("_", " "),
#                         href_decoded.replace("%20", " "),
#                         href_decoded.lower(),
#                         href_decoded.lower().replace("_", " ")
#                     ]:
#                         target_id = url_to_id.get(variation)
#                         if target_id:
#                             break
                
#                 if target_id and target_id != article_id:
#                     ground_truth_links.append({
#                         'anchor': anchor,
#                         'target_id': target_id,
#                         'href_decoded': href_decoded
#                     })
            
#             if len(ground_truth_links) >= min_links_per_sentence:
#                 test_data.append({
#                     'source_article_id': article_id,
#                     'source_article_title': article_title,
#                     'sentence': sentence,
#                     'ground_truth_links': ground_truth_links,
#                     'num_ground_truth': len(ground_truth_links)
#                 })
        
#         articles_processed += 1
    
#     print(f"\n✅ Created test dataset:")
#     print(f"   Sentences: {len(test_data)}")
#     print(f"   Articles processed: {articles_processed}")
#     total_gt_links = sum(t['num_ground_truth'] for t in test_data)
#     print(f"   Total ground truth links: {total_gt_links}")
    
#     return test_data


# # Create test dataset with position-based matching
# test_data = create_test_dataset_with_position_matching(
#     df_lazy,
#     url_to_id, 
#     max_sentences=10000000,
#     min_links_per_sentence=1,
#     sample_articles=1000
# )

🔍 Creating test dataset with position-based matching...
   Sampling 1,000 articles...


/tmp/ipykernel_2698231/569278588.py:147: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  .collect(streaming=True)


   Loaded 1,000 articles


Processing: 100%|██████████| 1000/1000 [00:04<00:00, 200.51it/s]


✅ Created test dataset:
   Sentences: 51956
   Articles processed: 1000
   Total ground truth links: 117577


In [8]:
def create_test_dataset_with_chunking(
    df_lazy: pl.LazyFrame,
    url_to_id: Dict[str, int],
    max_chunks: int = 100000,
    min_links_per_chunk: int = 1,
    sample_articles: int = 5000,
    chunking_strategy: str = "paragraph"  # "paragraph", "sentence", or "recursive"
) -> List[Dict]:
    """
    Create test dataset using paragraph/sentence-based chunking instead of 
    individual sentence extraction.
    
    Args:
        chunking_strategy: 
            - "paragraph": Split on paragraph breaks
            - "sentence": Group sentences up to max_words
            - "recursive": Try paragraphs first, then sentences
    """
    print(f"🔍 Creating test dataset with {chunking_strategy}-based chunking...")
    print(f"   Sampling {sample_articles:,} articles...")
    
    # Sample articles
    sampled_df = (
        df_lazy
        .filter(pl.col('link_count') > 0)
        .filter(pl.col('text').is_not_null())
        .filter(pl.col('text').str.len_chars() > 100)
        .select(['id', 'title', 'text'])
        .head(sample_articles * 3)
        .collect(streaming=True)
        .sample(n=min(sample_articles, sample_articles * 3), shuffle=True, seed=42)
    )
    
    print(f"   Loaded {len(sampled_df):,} articles")
    
    test_data = []
    articles_processed = 0
    
    for row in tqdm(sampled_df.iter_rows(named=True), total=len(sampled_df), desc="Processing"):
        if len(test_data) >= max_chunks:
            break
        
        article_id = row["id"]
        article_title = row["title"]
        raw_text = row.get("text", "")
        
        if not raw_text:
            continue
        
        # Extract links and get clean text
        clean_text, extracted_links = extract_links_and_clean_text(raw_text)
        
        if not clean_text or len(extracted_links) == 0:
            continue
        
        # Apply chunking strategy to clean text
        if chunking_strategy == "paragraph":
            chunks = paragraph_chunk(clean_text, min_length=50)
        elif chunking_strategy == "sentence":
            chunks = sentence_chunk(clean_text, max_words=150, min_words=20)
        elif chunking_strategy == "recursive":
            chunks = recursive_chunk(clean_text, chunk_size=300)
        else:
            raise ValueError(f"Unknown chunking strategy: {chunking_strategy}")
        
        # For each chunk, find links that fall within it
        current_pos = 0
        for chunk in chunks:
            if len(test_data) >= max_chunks:
                break
            
            # Find chunk position in clean text
            chunk_start = clean_text.find(chunk, current_pos)
            if chunk_start == -1:
                continue
            chunk_end = chunk_start + len(chunk)
            current_pos = chunk_end
            
            # Find links within this chunk's boundaries
            links_in_chunk = []
            for link in extracted_links:
                link_start = link.get('start_idx', -1)
                if chunk_start <= link_start < chunk_end:
                    links_in_chunk.append(link)
            
            if not links_in_chunk:
                continue
            
            # Map links to article IDs
            ground_truth_links = []
            for link in links_in_chunk:
                href_decoded = link["href_decoded"]
                anchor = link["anchor"]
                
                if not anchor or not anchor.strip():
                    continue
                
                target_id = url_to_id.get(href_decoded)
                if not target_id:
                    for variation in [
                        href_decoded.replace("_", " "),
                        href_decoded.replace("%20", " "),
                        href_decoded.lower(),
                        href_decoded.lower().replace("_", " ")
                    ]:
                        target_id = url_to_id.get(variation)
                        if target_id:
                            break
                
                if target_id and target_id != article_id:
                    ground_truth_links.append({
                        'anchor': anchor,
                        'target_id': target_id,
                        'href_decoded': href_decoded
                    })
            
            if len(ground_truth_links) >= min_links_per_chunk:
                test_data.append({
                    'source_article_id': article_id,
                    'source_article_title': article_title,
                    'sentence': chunk,  # Keep key name for compatibility
                    'chunk_type': chunking_strategy,
                    'ground_truth_links': ground_truth_links,
                    'num_ground_truth': len(ground_truth_links)
                })
        
        articles_processed += 1
    
    print(f"\n✅ Created test dataset:")
    print(f"   Chunks: {len(test_data)}")
    print(f"   Articles processed: {articles_processed}")
    total_gt_links = sum(t['num_ground_truth'] for t in test_data)
    print(f"   Total ground truth links: {total_gt_links}")
    avg_chunk_len = np.mean([len(t['sentence'].split()) for t in test_data]) if test_data else 0
    print(f"   Avg chunk length: {avg_chunk_len:.1f} words")
    
    return test_data


# Create test dataset with paragraph-based chunking
test_data = create_test_dataset_with_chunking(
    df_lazy,
    url_to_id, 
    max_chunks=100000,
    min_links_per_chunk=1,
    sample_articles=1000,
    chunking_strategy="paragraph"  # Try "sentence" or "recursive" too
)

🔍 Creating test dataset with paragraph-based chunking...
   Sampling 1,000 articles...


/tmp/ipykernel_3605256/1855920295.py:30: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  .collect(streaming=True)


   Loaded 1,000 articles


Processing: 100%|██████████| 1000/1000 [00:02<00:00, 431.39it/s]


✅ Created test dataset:
   Chunks: 25815
   Articles processed: 1000
   Total ground truth links: 106321
   Avg chunk length: 78.9 words


In [ ]:
# def predict_links_for_sentences(
#     client,
#     model,
#     test_data: List[Dict],
#     collection_name: str = "wikipedia_fr_chunks",
#     top_k: int = 10,
#     min_similarity: float = 0.5
# ) -> List[Dict]:
#     """
#     Predict links by finding similar articles in Qdrant.
    
#     Approach: Encode sentence → Find similar articles → Suggest those articles as links
#     """
#     print(f"🔮 Predicting links for {len(test_data)} sentences...")
    
#     # Extract all sentences
#     sentences = [t['sentence'] for t in test_data]
#     source_ids = [t['source_article_id'] for t in test_data]
    
#     # Encode all sentences
#     print("   Encoding sentences...")
#     embeddings = model.encode(
#         sentences,
#         normalize_embeddings=True,
#         show_progress_bar=True,
#         convert_to_numpy=True,
#         device=device
#     )
    
#     predictions = []
    
#     print("   Querying Qdrant...")
#     for i, (sentence, embedding, source_id) in enumerate(tqdm(
#         zip(sentences, embeddings, source_ids), 
#         total=len(sentences)
#     )):
#         # Search for similar articles
#         search_results = client.query_points(
#             collection_name=collection_name,
#             query=embedding.tolist(),
#             limit=top_k + 1,  # Extra to account for self-match
#             score_threshold=min_similarity
#         ).points
        
#         # Filter out source article and collect predictions
#         predicted_articles = []
#         for result in search_results:
#             article_id = result.payload.get('id')
            
#             # Skip self-match
#             if article_id == source_id:
#                 continue
            
#             predicted_articles.append({
#                 'article_id': article_id,
#                 'article_title': result.payload.get('title', f'ID {article_id}'),
#                 'similarity_score': result.score,
#                 'text_preview': result.payload.get('text_withoutHref', '')[:150] + '...'
#             })
        
#         # Limit to top_k after filtering
#         predicted_articles = predicted_articles[:top_k]
        
#         predictions.append({
#             **test_data[i],
#             'predicted_articles': predicted_articles,
#             'num_predictions': len(predicted_articles)
#         })
    
#     print(f"✅ Generated predictions for {len(predictions)} sentences")
#     return predictions

# # Generate predictions
# predictions = predict_links_for_sentences(
#     client,
#     model,
#     test_data,
#     collection_name="wikipedia_fr_chunks",
#     top_k=20,
#     min_similarity=0.5
# )

🔮 Predicting links for 25815 sentences...
   Encoding sentences...


Batches: 100%|██████████| 807/807 [05:26<00:00,  2.47it/s]


   Querying Qdrant...


100%|██████████| 25815/25815 [07:30<00:00, 57.33it/s]

✅ Generated predictions for 25815 sentences


In [9]:
def predict_links_for_sentences(
    client,
    model,
    test_data: List[Dict],
    collection_name: str = "wikipedia_fr_chunks",  # Updated default
    top_k: int = 10,
    min_similarity: float = 0.5
) -> List[Dict]:
    """
    Predict links by finding similar chunks in Qdrant, then aggregating by source article.
    """
    print(f"🔮 Predicting links for {len(test_data)} sentences...")
    
    # Extract all sentences
    sentences = [t['sentence'] for t in test_data]
    source_ids = [t['source_article_id'] for t in test_data]
    
    # Encode all sentences
    print("   Encoding sentences...")
    embeddings = model.encode(
        sentences,
        normalize_embeddings=True,
        show_progress_bar=True,
        convert_to_numpy=True,
        device=device
    )
    
    predictions = []
    
    print("   Querying Qdrant...")
    for i, (sentence, embedding, source_id) in enumerate(tqdm(
        zip(sentences, embeddings, source_ids), 
        total=len(sentences)
    )):
        # Search for similar chunks (not articles)
        search_results = client.query_points(
            collection_name=collection_name,
            query=embedding.tolist(),
            limit=top_k * 3,  # Get more chunks to aggregate by article
            score_threshold=min_similarity
        ).points
        
        # === FIX: Aggregate chunks by source article ===
        article_scores = {}
        for result in search_results:
            # Use correct field names for chunked collection
            article_id = result.payload.get('source_article_id')  # Changed from 'id'
            article_title = result.payload.get('source_article_title', f'ID {article_id}')  # Changed from 'title'
            
            # Skip self-match
            if article_id == source_id:
                continue
            
            # Aggregate scores by article (take max score across chunks)
            if article_id not in article_scores:
                article_scores[article_id] = {
                    'article_id': article_id,
                    'article_title': article_title,
                    'max_similarity_score': result.score,
                    'chunk_count': 1,
                    'best_chunk_preview': result.payload.get('text_preview', '')[:150] + '...'
                }
            else:
                # Update with better score if found
                if result.score > article_scores[article_id]['max_similarity_score']:
                    article_scores[article_id]['max_similarity_score'] = result.score
                    article_scores[article_id]['best_chunk_preview'] = result.payload.get('text_preview', '')[:150] + '...'
                article_scores[article_id]['chunk_count'] += 1
        
        # Sort by max similarity and take top_k articles
        predicted_articles = sorted(
            article_scores.values(), 
            key=lambda x: x['max_similarity_score'], 
            reverse=True
        )[:top_k]
        
        # Format for compatibility with existing evaluation code
        formatted_predictions = []
        for pred in predicted_articles:
            formatted_predictions.append({
                'article_id': pred['article_id'],
                'article_title': pred['article_title'],
                'similarity_score': pred['max_similarity_score'],
                'text_preview': pred['best_chunk_preview'],
                'chunk_count': pred['chunk_count']  # New: how many chunks matched
            })
        
        predictions.append({
            **test_data[i],
            'predicted_articles': formatted_predictions,
            'num_predictions': len(formatted_predictions)
        })
    
    print(f"✅ Generated predictions for {len(predictions)} sentences")
    return predictions

In [10]:
# Generate predictions using chunked collection
predictions = predict_links_for_sentences(
    client,
    model,
    test_data,
    collection_name="wikipedia_fr_chunks",  # Use chunked collection
    top_k=20,
    min_similarity=0.5
)

🔮 Predicting links for 25815 sentences...
   Encoding sentences...


Batches: 100%|██████████| 807/807 [10:59<00:00,  1.22it/s]


   Querying Qdrant...


100%|██████████| 25815/25815 [16:49<00:00, 25.58it/s]

✅ Generated predictions for 25815 sentences


In [11]:
def evaluate_predictions_recall_at_k(predictions: List[Dict], k_values: List[int] = [1, 5, 10, 20]) -> Dict:
    """
    Evaluate predictions using Recall@K metrics.
    
    Recall@K = What fraction of ground truth links appear in the top-K predictions?
    """
    # Initialize counters for each K
    recall_at_k = {k: {'hits': 0, 'total': 0} for k in k_values}
    
    results_per_sentence = []
    
    for pred in predictions:
        sentence_lower = pred['sentence'].lower()
        
        # Filter ground truth to only anchors actually in this sentence
        filtered_gt = [link for link in pred['ground_truth_links']
                       if link['anchor'].lower() in sentence_lower]
        
        gt_ids = set(link['target_id'] for link in filtered_gt)
        
        # Get predicted IDs in ranked order
        pred_ids_ranked = [p['article_id'] for p in pred['predicted_articles']]
        
        # Calculate Recall@K for each K value
        sentence_recall_at_k = {}
        for k in k_values:
            top_k_preds = set(pred_ids_ranked[:k])
            hits = len(gt_ids & top_k_preds)
            sentence_recall_at_k[k] = hits / len(gt_ids) if gt_ids else 0
            
            # Accumulate for overall metrics
            recall_at_k[k]['hits'] += hits
            recall_at_k[k]['total'] += len(gt_ids)
        
        results_per_sentence.append({
            **pred,
            'ground_truth_links': filtered_gt,
            'num_ground_truth': len(filtered_gt),
            'recall_at_k': sentence_recall_at_k
        })
    
    # Calculate overall Recall@K
    overall_recall_at_k = {}
    for k in k_values:
        if recall_at_k[k]['total'] > 0:
            overall_recall_at_k[k] = recall_at_k[k]['hits'] / recall_at_k[k]['total']
        else:
            overall_recall_at_k[k] = 0
    
    # Print results
    print("\n" + "="*60)
    print("📊 EVALUATION RESULTS (Recall@K)")
    print("="*60)
    print(f"\n📈 Recall@K Metrics:")
    for k in k_values:
        hits = recall_at_k[k]['hits']
        total = recall_at_k[k]['total']
        recall = overall_recall_at_k[k]
        print(f"   Recall@{k:2d}: {recall:.3f} ({hits:,}/{total:,})")
    
    print(f"\n📋 Dataset Stats:")
    print(f"   Test sentences: {len(predictions)}")
    total_gt = recall_at_k[k_values[0]]['total']
    print(f"   Total ground truth links: {total_gt:,}")
    
    # Also compute MRR (Mean Reciprocal Rank)
    mrr_sum = 0
    mrr_count = 0
    for pred in predictions:
        gt_ids = set(link['target_id'] for link in pred.get('ground_truth_links', [])
                     if link['anchor'].lower() in pred['sentence'].lower())
        pred_ids_ranked = [p['article_id'] for p in pred['predicted_articles']]
        
        for gt_id in gt_ids:
            mrr_count += 1
            if gt_id in pred_ids_ranked:
                rank = pred_ids_ranked.index(gt_id) + 1
                mrr_sum += 1.0 / rank
    
    mrr = mrr_sum / mrr_count if mrr_count > 0 else 0
    print(f"\n📊 Mean Reciprocal Rank (MRR): {mrr:.3f}")
    
    return {
        'recall_at_k': overall_recall_at_k,
        'mrr': mrr,
        'results_per_sentence': results_per_sentence
    }

# Evaluate with Recall@K
eval_results = evaluate_predictions_recall_at_k(predictions, k_values=[1, 5, 10, 20])


📊 EVALUATION RESULTS (Recall@K)

📈 Recall@K Metrics:
   Recall@ 1: 0.061 (6,372/104,555)
   Recall@ 5: 0.163 (17,074/104,555)
   Recall@10: 0.219 (22,939/104,555)
   Recall@20: 0.283 (29,560/104,555)

📋 Dataset Stats:
   Test sentences: 25815
   Total ground truth links: 104,555

📊 Mean Reciprocal Rank (MRR): 0.109


In [12]:
def display_detailed_analysis(eval_results: Dict, num_samples: int = 5, k_display: int = 5):
    """Show detailed examples of predictions vs ground truth using Recall@K."""
    
    results = eval_results['results_per_sentence']
    
    # Sort by Recall@k_display to show best and worst
    sorted_results = sorted(results, key=lambda x: x['recall_at_k'].get(k_display, 0), reverse=True)
    
    print("\n" + "="*80)
    print(f"🎯 BEST PREDICTIONS (Highest Recall@{k_display})")
    print("="*80)
    
    for i, result in enumerate(sorted_results[:num_samples]):
        recall_k = result['recall_at_k'].get(k_display, 0)
        if recall_k == 0:
            continue
        print(f"\n{'─'*80}")
        print(f"Example {i+1}: Article '{result['source_article_title']}'")
        print(f"Sentence: '{result['sentence']}'")
        print(f"Recall@1: {result['recall_at_k'].get(1, 0):.2f} | Recall@5: {result['recall_at_k'].get(5, 0):.2f} | Recall@10: {result['recall_at_k'].get(10, 0):.2f}")
        
        # Get ground truth IDs and predicted IDs
        gt_ids = set(gt['target_id'] for gt in result['ground_truth_links'])
        pred_ids_ranked = [p['article_id'] for p in result['predicted_articles']]
        top_k_pred_ids = set(pred_ids_ranked[:k_display])
        
        print(f"\n✅ Ground Truth ({len(result['ground_truth_links'])}):")
        for gt in result['ground_truth_links']:
            target_title = id_to_title.get(gt['target_id'], f"ID {gt['target_id']}")
            matched = "✓" if gt['target_id'] in top_k_pred_ids else " "
            # Show rank if found
            if gt['target_id'] in pred_ids_ranked:
                rank = pred_ids_ranked.index(gt['target_id']) + 1
                print(f"   [{matched}] '{gt['anchor']}' → {target_title} (rank: {rank})")
            else:
                print(f"   [{matched}] '{gt['anchor']}' → {target_title} (not in top {len(pred_ids_ranked)})")
        
        print(f"\n🔮 Top {min(k_display, result['num_predictions'])} Predictions:")
        for j, pred in enumerate(result['predicted_articles'][:k_display]):
            matched = "✓" if pred['article_id'] in gt_ids else "✗"
            print(f"   [{matched}] #{j+1} {pred['article_title']} (sim: {pred['similarity_score']:.3f})")
    
    print("\n" + "="*80)
    print(f"❌ WORST PREDICTIONS (Lowest Recall@{k_display}, with predictions)")
    print("="*80)
    
    worst = [r for r in sorted_results if r['num_predictions'] > 0][-num_samples:]
    
    for i, result in enumerate(worst):
        print(f"\n{'─'*80}")
        print(f"Example {i+1}: Article '{result['source_article_title']}'")
        print(f"Sentence: '{result['sentence']}'")
        print(f"Recall@1: {result['recall_at_k'].get(1, 0):.2f} | Recall@5: {result['recall_at_k'].get(5, 0):.2f} | Recall@10: {result['recall_at_k'].get(10, 0):.2f}")
        
        gt_ids = set(gt['target_id'] for gt in result['ground_truth_links'])
        pred_ids_ranked = [p['article_id'] for p in result['predicted_articles']]
        
        print(f"\n✅ Ground Truth ({len(result['ground_truth_links'])}):")
        for gt in result['ground_truth_links']:
            target_title = id_to_title.get(gt['target_id'], f"ID {gt['target_id']}")
            if gt['target_id'] in pred_ids_ranked:
                rank = pred_ids_ranked.index(gt['target_id']) + 1
                print(f"   '{gt['anchor']}' → {target_title} (rank: {rank})")
            else:
                print(f"   '{gt['anchor']}' → {target_title} (not found)")
        
        print(f"\n🔮 Top {min(k_display, result['num_predictions'])} Predictions:")
        for j, pred in enumerate(result['predicted_articles'][:k_display]):
            matched = "✓" if pred['article_id'] in gt_ids else "✗"
            print(f"   [{matched}] #{j+1} {pred['article_title']} (sim: {pred['similarity_score']:.3f})")

# Display analysis
display_detailed_analysis(eval_results, num_samples=20, k_display=5)


🎯 BEST PREDICTIONS (Highest Recall@5)

────────────────────────────────────────────────────────────────────────────────
Example 1: Article 'Les Verts (France)'
Sentence: 'Lors des élections régionales de 1992, la région Nord-Pas-de-Calais voit élue à la Présidence de son conseil régional une femme – la première en France – adhérente des Verts - Marie-Christine Blandin, après un accord avec le P.S qui avait une majorité relative en nombre de conseillers régionaux.'
Recall@1: 0.50 | Recall@5: 1.00 | Recall@10: 1.00

✅ Ground Truth (2):
   [✓] 'élections régionales de 1992' → Élections régionales françaises de 1992 (rank: 2)
   [✓] 'Marie-Christine Blandin' → Marie-Christine Blandin (rank: 1)

🔮 Top 5 Predictions:
   [✓] #1 Marie-Christine Blandin (sim: 0.893)
   [✓] #2 Élections régionales françaises de 1992 (sim: 0.872)
   [✗] #3 Alliances électorales du Front national (sim: 0.872)
   [✗] #4 Liste des membres du conseil régional du Nord-Pas-de-Calais (1998-2004) (sim: 0.871)
   [✗] #5 

In [13]:
def print_summary_statistics(eval_results: Dict):
    """Print summary statistics about the evaluation using Recall@K."""
    
    results = eval_results['results_per_sentence']
    
    # Count sentences with at least one correct prediction in top-K
    sentences_with_hit_at_1 = sum(1 for r in results if r['recall_at_k'].get(1, 0) > 0)
    sentences_with_hit_at_5 = sum(1 for r in results if r['recall_at_k'].get(5, 0) > 0)
    sentences_with_hit_at_10 = sum(1 for r in results if r['recall_at_k'].get(10, 0) > 0)
    
    # Average Recall@K per sentence
    avg_recall_at_1 = np.mean([r['recall_at_k'].get(1, 0) for r in results])
    avg_recall_at_5 = np.mean([r['recall_at_k'].get(5, 0) for r in results])
    avg_recall_at_10 = np.mean([r['recall_at_k'].get(10, 0) for r in results])
    avg_recall_at_20 = np.mean([r['recall_at_k'].get(20, 0) for r in results])
    
    # Distribution of predictions
    pred_counts = [r['num_predictions'] for r in results]
    gt_counts = [r['num_ground_truth'] for r in results]
    
    print("\n" + "="*60)
    print("📊 SUMMARY STATISTICS (Recall@K)")
    print("="*60)
    
    print(f"\n🎯 Hit Rate (sentences with ≥1 correct in top-K):")
    print(f"   Hit@1:  {sentences_with_hit_at_1}/{len(results)} ({100*sentences_with_hit_at_1/len(results):.1f}%)")
    print(f"   Hit@5:  {sentences_with_hit_at_5}/{len(results)} ({100*sentences_with_hit_at_5/len(results):.1f}%)")
    print(f"   Hit@10: {sentences_with_hit_at_10}/{len(results)} ({100*sentences_with_hit_at_10/len(results):.1f}%)")
    
    print(f"\n📈 Average Per-Sentence Recall@K:")
    print(f"   Avg Recall@1:  {avg_recall_at_1:.3f}")
    print(f"   Avg Recall@5:  {avg_recall_at_5:.3f}")
    print(f"   Avg Recall@10: {avg_recall_at_10:.3f}")
    print(f"   Avg Recall@20: {avg_recall_at_20:.3f}")
    
    print(f"\n📊 Overall Metrics:")
    print(f"   MRR: {eval_results['mrr']:.3f}")
    for k, recall in eval_results['recall_at_k'].items():
        print(f"   Recall@{k}: {recall:.3f}")
    
    print(f"\n📋 Prediction Distribution:")
    print(f"   Avg predictions per sentence: {np.mean(pred_counts):.1f}")
    print(f"   Avg ground truth per sentence: {np.mean(gt_counts):.1f}")
    
    print(f"\n🔍 Coverage:")
    print(f"   Sentences with 0 predictions: {sum(1 for p in pred_counts if p == 0)}")
    print(f"   Sentences with 10+ predictions: {sum(1 for p in pred_counts if p >= 10)}")

print_summary_statistics(eval_results)


📊 SUMMARY STATISTICS (Recall@K)

🎯 Hit Rate (sentences with ≥1 correct in top-K):
   Hit@1:  6372/25815 (24.7%)
   Hit@5:  12330/25815 (47.8%)
   Hit@10: 14550/25815 (56.4%)

📈 Average Per-Sentence Recall@K:
   Avg Recall@1:  0.092
   Avg Recall@5:  0.216
   Avg Recall@10: 0.277
   Avg Recall@20: 0.341

📊 Overall Metrics:
   MRR: 0.109
   Recall@1: 0.061
   Recall@5: 0.163
   Recall@10: 0.219
   Recall@20: 0.283

📋 Prediction Distribution:
   Avg predictions per sentence: 20.0
   Avg ground truth per sentence: 4.1

🔍 Coverage:
   Sentences with 0 predictions: 0
   Sentences with 10+ predictions: 25815
